In [ ]:
'''
Creating a RNNs model with pre-trained GloVe word vectors
'''

In [230]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
import random
import sys, os, subprocess, time, re

In [231]:
# max number of words in each sentence
SEQUENCE_LENGTH = 100
# N-Dimensional GloVe embedding vectors
EMBEDDING_SIZE = 300
# number of words to use, discarding the rest
N_WORDS = 10000
# out of vocabulary token
OOV_TOKEN = None
# 25% testing set, 75% training set
TEST_SIZE = 0.25
# number of CELL layers
N_LAYERS = 1
# the RNN cell to use, LSTM in this case
RNN_CELL = LSTM
# whether it's a bidirectional RNN
IS_BIDIRECTIONAL = True
# number of units (RNN_CELL ,nodes) in each layer
UNITS = 128
# dropout rate
DROPOUT = 0.4
### Training parameters
LOSS = "categorical_crossentropy"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 6

In [232]:
df = pd.read_csv('final_dataset.csv')

In [233]:
dt = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, "H": 7}
df.IPC = df.IPC.map(dt)

In [234]:
tokenizer = Tokenizer(num_words=N_WORDS, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(df.content)

In [235]:
X = tokenizer.texts_to_sequences(df.content)

In [236]:
X, y = np.array(X), np.array(df.IPC)

<ipython-input-236-d93bec3727fe>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X, y = np.array(X), np.array(df.IPC)


In [237]:
X = pad_sequences(X, maxlen=SEQUENCE_LENGTH)

In [238]:
y = to_categorical(y)

In [239]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=1)

In [240]:
data = {}
data["X_train"] = X_train
data["X_test"]= X_test
data["y_train"] = y_train
data["y_test"] = y_test
data["tokenizer"] = tokenizer

In [241]:
def get_embedding_vectors(word_index, embedding_size=300):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    with open(f"data/glove.6B.{embedding_size}d.txt", encoding="utf8") as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            # get the word as the first word in the line
            word = values[0]
            if word in word_index:
                idx = word_index[word]
                # get the vectors as the remaining values in the line
                embedding_matrix[idx] = np.array(values[1:], dtype="float32")
    return embedding_matrix

In [242]:
def create_model(word_index, units=128, n_layers=1, cell=LSTM, bidirectional=False,
                embedding_size=100, sequence_length=100, dropout=0.3, 
                loss="categorical_crossentropy", optimizer="adam", 
                output_length=8):
    """Constructs a RNN model given its parameters"""
    embedding_matrix = get_embedding_vectors(word_index, embedding_size)
    model = Sequential()
    # add the embedding layer
    model.add(Embedding(len(word_index) + 1,
              embedding_size,
              weights=[embedding_matrix],
              trainable=False,
              input_length=sequence_length))
    for i in range(n_layers):
        if i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # first layer or hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        model.add(Dropout(dropout))
    model.add(Dense(output_length, activation="softmax"))
    # compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
    return model

In [ ]:
### pre-trained GloVe would need to be downloaded and placed under './data' folder

In [243]:
def get_model_name(dataset_name):
    # construct the unique model name
    model_name = f"{dataset_name}-{RNN_CELL.__name__}-seq-{SEQUENCE_LENGTH}-em-{EMBEDDING_SIZE}-w-{N_WORDS}-layers-{N_LAYERS}-units-{UNITS}-opt-{OPTIMIZER}-BS-{BATCH_SIZE}-d-{DROPOUT}"
    if IS_BIDIRECTIONAL:
        # add 'bid' str if bidirectional
        model_name = "bid-" + model_name
    if OOV_TOKEN:
        # add 'oov' str if OOV token is specified
        model_name += "-oov"
    return model_name

In [244]:
model = create_model(data["tokenizer"].word_index, units=UNITS, n_layers=N_LAYERS, 
                    cell=RNN_CELL, bidirectional=IS_BIDIRECTIONAL, embedding_size=EMBEDDING_SIZE, 
                    sequence_length=SEQUENCE_LENGTH, dropout=DROPOUT, 
                    loss=LOSS, optimizer=OPTIMIZER, output_length=8)
model.summary()

Reading GloVe: 400000it [00:09, 40461.89it/s]


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 100, 300)          21114900  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 2056      
Total params: 21,556,252
Trainable params: 441,352
Non-trainable params: 21,114,900
_________________________________________________________________


In [245]:
dataset_name = "e-OG20210223_1483-4"
model_name = get_model_name(dataset_name)
# using tensorboard on 'logs' folder

tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))
# start training
history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[tensorboard],
                    verbose=1)
# save the resulting model into 'results' folder
model.save(os.path.join("results", model_name) + ".h5")

Epoch 1/6
85/85 [==============================] - 49s 532ms/step - loss: 1.6105 - accuracy: 0.3608 - val_loss: 1.2821 - val_accuracy: 0.5014
Epoch 2/6
85/85 [==============================] - 41s 484ms/step - loss: 1.2657 - accuracy: 0.5004 - val_loss: 1.2209 - val_accuracy: 0.5452
Epoch 3/6
85/85 [==============================] - 51s 603ms/step - loss: 1.1297 - accuracy: 0.5853 - val_loss: 1.2177 - val_accuracy: 0.5419
Epoch 4/6
85/85 [==============================] - 60s 712ms/step - loss: 1.0590 - accuracy: 0.6003 - val_loss: 1.1773 - val_accuracy: 0.5602
Epoch 5/6
85/85 [==============================] - 67s 786ms/step - loss: 0.9569 - accuracy: 0.6378 - val_loss: 1.1935 - val_accuracy: 0.5630
Epoch 6/6
85/85 [==============================] - 55s 648ms/step - loss: 0.8917 - accuracy: 0.6769 - val_loss: 1.1726 - val_accuracy: 0.5624


In [ ]:
### Training Accuracy is 67.7 % and the validation accuracy is 56.2%